<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to visualize iMOD inputs in QGIS.
</span><br>

In [1]:
print(f"\033[1m{'*'*50} Mdl_In_to_mm {'*'*50}\033[0m")
print('This script converts model inputs (mainly IDF) to TIF files, to be visualized in QGIS. It also calculates some parameters from Mdl Ins (e.g.) Aquifer layer thickness is calculated from TOP-BOT (same as MF does internally).')

************************************************** Mdl_In_to_mm **************************************************
This script converts model inputs (mainly IDF) to TIF files, to be visualized in QGIS. It also calculates some parameters from Mdl Ins (e.g.) Aquifer layer thickness is calculated from TOP-BOT (same as MF does internally).


# 0. Prep

## 0.0. Libraries

In [2]:
from WS_Mdl import utils as U
from WS_Mdl import geo as G
from WS_Mdl import utils_imod as UIM

In [3]:
import importlib
importlib.reload(G)
importlib.reload(U)
importlib.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'c:\\users\\karam014\\onedrive - universiteit utrecht\\ws_mdl\\code\\WS_Mdl\\utils_imod.py'>

## 0.1. Options

In [4]:
MdlN = 'NBr13'

# 1. PoP In
Also makes replace.csv for S QGIS project.

In [ ]:
G.PRJ_to_TIF(MdlN)

# 2. Make new MM

## 2.0 Init

In [ ]:
G.Up_MM(MdlN)

# 3. PoP Out

## 3.0. GXG

### 3.0.1. Write TIF files

In [ ]:
G.HD_IDF_GXG_to_MBTIF(MdlN)

In [ ]:
G.HD_IDF_GXG_to_MBTIF('NBr12')

### 3.0.1 Calculate differences

In [ ]:
import os
import rasterio
import numpy as np

In [ ]:
d_paths = U.get_MdlN_paths(MdlN)
MdlN_B, path_PoP, path_MdlN, path_PoP_Out_MdlN, path_PoP_Out_MdlN_B = [ d_paths[v] for v in ['MdlN_B', 'path_PoP', 'path_MdlN', 'path_PoP_Out_MdlN', 'path_PoP_Out_MdlN_B'] ]

In [ ]:
path_GXG = os.path.join(path_PoP_Out_MdlN, 'GXG')
path_GXG_B = path_GXG.replace (MdlN, MdlN_B)
path_GXG_SmB = os.path.join(path_PoP_Out_MdlN, 'GXG_SmB')
os.makedirs(path_GXG_SmB, exist_ok=True)

In [ ]:
l_path_S = [os.path.join(path_GXG, p) for p in sorted(os.listdir(path_GXG), key=lambda f: int(f.split('_')[1][1:]))]
l_path_B = [x.replace(MdlN, MdlN_B) for x in l_path_S]

In [ ]:
for p_S, p_B in zip(l_path_S, l_path_B):
    # print(f_S, f_B, '-'*50, sep='\n')
    with rasterio.open(p_S) as f_S, rasterio.open(p_B) as f_B:
        assert f_S.count == f_B.count, f"Different number of bands:\n{p_S}:\t{f_S.count}\n{p_B}:\t{f_B.count}"
        assert f_S.width == f_B.width and f_S.height == f_B.height, f"Image dimensions must match:\n{p_S}:\t{f_S.width}x{f_S.height}\n{p_B}:\t{f_B.width}x{f_B.height}"

        profile = f_S.profile  # Use metadata from the first file
        profile.update(dtype=rasterio.float32)  # Ensure output can handle differences (including negatives)

        path_Out = os.path.join(path_GXG_SmB, os.path.basename(p_S).replace(MdlN, f"{MdlN}_m_{MdlN_B}"))
        print(path_Out)

        with rasterio.open(path_Out, 'w', **profile) as dst:
            for i in range(f_S.count):
                d_S = f_S.read(i + 1).astype(np.float32)
                d_B = f_B.read(i + 1).astype(np.float32)
                dst.write(d_S - d_B, i + 1)

            tags_S = f_S.tags()
            tags_B = f_B.tags()
            combined_tags = {**{f"S_{k}": v for k, v in tags_S.items()},
                             **{f"B_{k}": v for k, v in tags_B.items()}}
            dst.update_tags(**combined_tags)

## 3.1. Other PoP

In [14]:
import re
import os
from pathlib import Path
import imod
N_cores = None

In [28]:
DF_rules = "(L == 1)"

In [29]:
# Get paths
d_paths = U.get_MdlN_paths(MdlN)
path_PoP, path_MdlN = [ d_paths[v] for v in ['path_PoP', 'path_MdlN'] ]
path_HD = os.path.join(path_MdlN, 'GWF_1/MODELOUTPUT/HEAD/HEAD')

✅ - NBr13 paths extracted from RunLog and returned as dictionary with keys:
Mdl, MdlN_B, path_Mdl, path_INI, path_BAT, path_PRJ, path_Smk, path_MdlN, path_Out_HD, path_PoP, path_PoP_Out_MdlN, path_MM, path_INI_B, path_BAT_B, path_PRJ_B, path_Smk_B, path_MdlN_B, path_Out_HD_B, path_PoP_Out_MdlN_B, path_MM_B


In [30]:
DF = U.HD_Out_IDF_to_DF(path_HD) # Read the IDF files to a DataFrame

In [31]:
if DF_rules is not None:
    DF = DF.query(DF_rules)


In [ ]:
# Continue from here

In [ ]:
# Get list of layers in the model
l_L = sorted({int(match.group(1)) for f in Path(path_HD).glob("HEAD_*.IDF")
            if (match := re.compile(r"_L(\d+)\.IDF$").search(f.name))})

In [16]:
# Make a dictionary of the IDF files for each layer
d_IDF_GXG = {i: sorted(f for f in Path(path_HD).glob(f"HEAD_*_L{i}.IDF")
                    if re.search(r'HEAD_(\d{4})(\d{2})(\d{2})', f.name)
                    and int((m := re.search(r'HEAD_(\d{4})(\d{2})(\d{2})', f.name)).group(3)) in {14, 28})
            for i in l_L}

In [ ]:
if N_cores is None:
    N_cores = max(os.cpu_count() - 2, 1)
start = DT.now() # Start time

start = DT.now()
with PPE(max_workers=N_cores) as E:
    futures = [E.submit(_HD_IDF_GXG_to_MBTIF_process_L, L, d_IDF_GXG, MdlN, path_PoP, path_HD, crs)
                for L in d_IDF_GXG.keys()]
    for f in futures:
        print('\t', f.result(), '- Elapsed time (from start):', DT.now() - start)

print('Total elapsed:', DT.now() - start)

In [ ]:

def _HD_IDF_GXG_to_MBTIF_process_L(L, d_IDF_GXG, MdlN, path_PoP, path_HD, crs):
    """Only for use within HD_IDF_GXG_to_MBTIF - to utilize multiprocessing."""
    XA = imod.idf.open(d_IDF_GXG[L])
    GXG = imod.evaluate.calculate_gxg(XA.squeeze())
    GXG = GXG.rename_vars({var: var.upper() for var in GXG.data_vars})
    GXG = GXG.rename_vars({'N_YEARS_GXG': 'N_years_GXG', 'N_YEARS_GVG': 'N_years_GVG'})
    GXG["GHG_m_GLG"] = GXG["GHG"] - GXG["GLG"]
    GXG = GXG[["GHG", "GLG", "GHG_m_GLG", "GVG", "N_years_GXG", "N_years_GVG"]]

    path_Out = os.path.join(path_PoP, 'Out', MdlN, 'GXG', f'GXG_L{L}_{MdlN}.tif')
    os.makedirs(os.path.dirname(path_Out), exist_ok=True)

    d_MtDt = {str(i+1): {f'{var}_AVG': float(GXG[var].mean().values) for var in GXG.data_vars} 
              for i in range(len(GXG.data_vars))}
    
    d_MtDt['all'] = {'parameters': XA.coords,
                     'Description': f'{MdlN} GXG (path: {path_HD})\nFor more info see: https://deltares.github.io/imod-python/api/generated/evaluate/imod.evaluate.calculate_gxg.html'}

    # Set proper band names and write to MBTIF
    band_names = [f"{var}_{MdlN}" for var in GXG.data_vars]
    DA = GXG.to_array(dim="band").astype(np.float32)
    DA["band"] = band_names
    DA_to_MBTIF(DA, path_Out, d_MtDt, crs=crs, _print=False)
    return f"GXG_L{L} ✔"

